In [1]:
#%cd /content/drive/My Drive/SegFormer
import argparse
import logging
import os
import random
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from utils import clip_gradient, AvgMeter

from glob import glob
from skimage.io import imread
import matplotlib.pyplot as plt
import pandas as pd
from collections import OrderedDict
from torch.autograd import Variable
from datetime import datetime
import torch.nn.functional as F
import cv2
from albumentations.augmentations import transforms
from albumentations.core.composition import Compose, OneOf
import math
import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

/home/s/anaconda3/envs/tf2.6/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import argparse
import copy
import os
import os.path as osp
import time

import mmcv
import torch
from mmcv.runner import init_dist
from mmcv.utils import Config, DictAction, get_git_hash

from mmseg import __version__
from mmseg.apis import set_random_seed, train_segmentor
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import collect_env, get_root_logger

In [3]:
from mmseg.models.segmentors import CaraSegUPer_ver2 as UNet
from mmseg.models.segmentors import CaraSegUPer_wBiFPN as Net
from mmseg.models.segmentors import CaraSegUPer_wBiRAFPN as Net2

In [4]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, img_paths, mask_paths, aug=True, transform=None):
        self.img_paths = img_paths
        self.mask_paths = mask_paths
        self.aug = aug
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        mask_path = self.mask_paths[idx]
        # image = imread(img_path)
        # mask = imread(mask_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, 0)
        # name = self.img_paths[idx].split('/')[-1]

        if self.transform is not None:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
            image = cv2.resize(image, (384, 384))
            mask = cv2.resize(mask, (384, 384)) 
        else:
            image = cv2.resize(image, (384, 384))
            mask = cv2.resize(mask, (384, 384)) 

        image = image.astype('float32') / 255
        image = image.transpose((2, 0, 1))

        mask = mask[:,:,np.newaxis]
        mask = mask.astype('float32') / 255
        mask = mask.transpose((2, 0, 1))

        return np.asarray(image), np.asarray(mask)

In [5]:
from keras import backend as K

def recall_m(y_true, y_pred):
  true_positives = torch.sum(torch.round(torch.clip(y_true * y_pred, 0, 1)))
  possible_positives = torch.sum(torch.round(torch.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision_m(y_true, y_pred):
  true_positives = torch.sum(torch.round(torch.clip(y_true * y_pred, 0, 1)))
  predicted_positives = torch.sum(torch.round(torch.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def dice_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

def iou_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return recall*precision/(recall+precision-recall*precision +K.epsilon())

In [6]:
class FocalLossV1(nn.Module):

    def __init__(self,
                 alpha=0.25,
                 gamma=2,
                 reduction='mean',):
        super(FocalLossV1, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.crit = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, logits, label):
        # compute loss
        logits = logits.float() # use fp32 if logits is fp16
        with torch.no_grad():
            alpha = torch.empty_like(logits).fill_(1 - self.alpha)
            alpha[label == 1] = self.alpha

        probs = torch.sigmoid(logits)
        pt = torch.where(label == 1, probs, 1 - probs)
        ce_loss = self.crit(logits, label.float())
        loss = (alpha * torch.pow(1 - pt, self.gamma) * ce_loss)
        if self.reduction == 'mean':
            loss = loss.mean()
        if self.reduction == 'sum':
            loss = loss.sum()
        return loss
class TverskyLoss(nn.Module):
    def __init__(self, alpha=0.7, eps=1):
        super().__init__()

        self.eps = eps
        self.alpha = alpha

    def forward(self, pred, label):
        """
        Forward function
        :param pred: Prediction tensor containing raw network outputs (no logit) (B x C x H x W)
        :param label: Label mask tensor (B x C x H x W)
        """
        probs = torch.sigmoid(pred)
        true_pos = torch.sum(probs * label, dim=[0, 2, 3])
        false_neg = torch.sum(label * (1 - probs), dim=[0, 2, 3])
        false_pos = torch.sum(probs * (1 - label), dim=[0, 2, 3])
        return 1 - torch.mean(
            (true_pos + self.eps)
            / (
                true_pos
                + self.alpha * false_neg
                + (1 - self.alpha) * false_pos
                + self.eps
            )
        )

class FocalTverskyLoss(TverskyLoss):
    def __init__(self, gamma=4 / 3, **kwargs):
        super().__init__(**kwargs)
        self.gamma = gamma

    def forward(self, pred, label):
        probs = torch.sigmoid(pred)
        true_pos = torch.sum(probs * label, dim=[0, 2, 3])
        false_neg = torch.sum(label * (1 - probs), dim=[0, 2, 3])
        false_pos = torch.sum(probs * (1 - label), dim=[0, 2, 3])

        t = (true_pos + self.eps) / (
            true_pos + self.alpha * false_neg + (1 - self.alpha) * false_pos + self.eps
        )

        x = torch.pow(1 - t, 1 / self.gamma)

        return x #torch.sum(x)
def structure_loss_v2(pred, mask):
    weit = 1 + 5*torch.abs(F.avg_pool2d(mask, kernel_size=31, stride=1, padding=15) - mask)
    wfocal = FocalLossV1()(pred, mask)
    wfocal = (wfocal*weit).sum(dim=(2,3)) / weit.sum(dim=(2, 3))

    
    wiou = FocalTverskyLoss()(pred,mask)
    return (wfocal + wiou).mean()
def structure_loss(pred, mask):
    weit = 1 + 5*torch.abs(F.avg_pool2d(mask, kernel_size=31, stride=1, padding=15) - mask)
    wfocal = FocalLossV1()(pred, mask)
    wfocal = (wfocal*weit).sum(dim=(2,3)) / weit.sum(dim=(2, 3))

    pred = torch.sigmoid(pred)
    inter = ((pred * mask)*weit).sum(dim=(2, 3))
    union = ((pred + mask)*weit).sum(dim=(2, 3))
    wiou = 1 - (inter + 1)/(union - inter+1)
    return (wfocal + wiou).mean()

In [7]:
def train(train_loader, model, optimizer, epoch, lr_scheduler, deep=False):
    model.train()
    # ---- multi-scale training ----
    size_rates = [256/384, 1, 512/384]
    loss_record = AvgMeter()
    dice, iou = AvgMeter(), AvgMeter()
    for i, pack in enumerate(train_loader, start=1):
        if epoch <= 1:
                optimizer.param_groups[0]["lr"] = (epoch * i) / (1.0 * total_step) * init_lr
        else:
            lr_scheduler.step()

        for rate in size_rates: 
            optimizer.zero_grad()
            # ---- data prepare ----
            images, gts = pack
            images = Variable(images).cuda()
            gts = Variable(gts).cuda()
            # ---- rescale ----
            trainsize = int(round(trainsize_init*rate/32)*32)
            images = F.upsample(images, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            gts = F.upsample(gts, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            # ---- forward ----
            map4, map3, map2, map1 = model(images)
           # print(map4.shape, map3.shape, map2.shape, map1.shape)
            map1 = F.upsample(map1, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            map2 = F.upsample(map2, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            map3 = F.upsample(map3, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            map4 = F.upsample(map4, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            loss = structure_loss(map1, gts) + structure_loss(map2, gts) + structure_loss(map3, gts) + structure_loss(map4, gts)
            # ---- metrics ----
            dice_score = dice_m(map2, gts)
            iou_score = iou_m(map2, gts)
            # ---- backward ----
            loss.backward()
            clip_gradient(optimizer, clip)
            optimizer.step()
            # ---- recording loss ----
            if rate == 1:
                loss_record.update(loss.data, batchsize)
                dice.update(dice_score.data, batchsize)
                iou.update(iou_score.data, batchsize)

        # ---- train visualization ----
        if i == total_step:
            print('{} Training Epoch [{:03d}/{:03d}], '
                  '[loss: {:0.4f}, dice: {:0.4f}, iou: {:0.4f}]'.
                  format(datetime.now(), epoch, num_epochs,\
                         loss_record.show(), dice.show(), iou.show()))

    ckpt_path = save_path + 'last.pth'
    print('[Saving Checkpoint:]', ckpt_path)
    checkpoint = {
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': lr_scheduler.state_dict()
    }
    torch.save(checkpoint, ckpt_path)

    log = OrderedDict([
        ('loss', loss_record.show()), ('dice', dice.show()), ('iou', iou.show()),
    ])

    return log

In [8]:
def recall_np(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_np(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    predicted_positives = np.sum(np.round(np.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def dice_np(y_true, y_pred):
    precision = precision_np(y_true, y_pred)
    recall = recall_np(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def iou_np(y_true, y_pred):
    intersection = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    union = np.sum(y_true)+np.sum(y_pred)-intersection
    return intersection/(union+K.epsilon())

def get_scores(gts, prs):
    mean_precision = 0
    mean_recall = 0
    mean_iou = 0
    mean_dice = 0
    for gt, pr in zip(gts, prs):
        mean_precision += precision_np(gt, pr)
        mean_recall += recall_np(gt, pr)
        mean_iou += iou_np(gt, pr)
        mean_dice += dice_np(gt, pr)

    mean_precision /= len(gts)
    mean_recall /= len(gts)
    mean_iou /= len(gts)
    mean_dice /= len(gts)        
    
    print(f"scores: dice={mean_dice}, miou={mean_iou}, precision={mean_precision}, recall={mean_recall}")

    return (mean_iou, mean_dice, mean_precision, mean_recall)



def inference(model,writer=None,epoch=None):
    print("#"*20)
    model.eval()
    dataset_names = ['Kvasir', 'CVC-ClinicDB', 'CVC-ColonDB', 'CVC-300', 'ETIS-LaribPolypDB']
    for dataset_name in dataset_names:
        data_path = f'../dataset/scenario_4/all_datasets//TestDataset/{dataset_name}'
        print(data_path)
        X_test = glob('{}/images/*'.format(data_path))
        X_test.sort()
        y_test = glob('{}/masks/*'.format(data_path))
        y_test.sort()

        test_dataset = Dataset(X_test, y_test)
        test_loader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=1,
            shuffle=False,
            pin_memory=True,
            drop_last=False)

        print('Dataset_name:', dataset_name)
        tp_all = 0
        fp_all = 0
        fn_all = 0
        mean_iou = 0
        gts = []
        prs = []
        losses = []
        for i, pack in enumerate(test_loader, start=1):
            image, gt_ = pack
            # name = name[0]
            gt = gt_[0][0]
            gt = np.asarray(gt, np.float32)
            image = image.cuda()

            res, res2, res3, res4 = model(image)
            res = F.upsample(res, size=gt.shape, mode='bilinear', align_corners=False)
            loss = structure_loss_v2(res.cpu(), gt_)
            losses.append(loss.cpu().detach().numpy().squeeze())
            res = res.sigmoid().data.cpu().numpy().squeeze()
            res = (res - res.min()) / (res.max() - res.min() + 1e-8)
            pr = res.round()
            # cv2.imwrite(os.path.join(save_path, dataset_name, name), res)
            gts.append(gt)
            prs.append(pr)
        mean_iou, mean_dice, mean_precision, mean_recall = get_scores(gts, prs)
        writer.add_scalar(dataset_name + ' val loss',
                            np.mean(losses),
                            epoch )
        writer.add_scalar(dataset_name + ' val dice',
                            mean_dice,
                            epoch )
        writer.add_scalar(dataset_name + ' val iou',
                            mean_iou,
                            epoch )
    print("#"*20)

In [9]:
#from torchvision.transforms.functional import *
#import torchvision.transforms as transforms
from albumentations.augmentations.geometric import  resize,rotate
import albumentations.augmentations.crops.transforms as crop
import albumentations.augmentations.transforms as transforms
train_transform = Compose([
            rotate.RandomRotate90(),
            transforms.Flip(),
            transforms.HueSaturationValue(),
            transforms.RandomBrightnessContrast(),
            transforms.GaussianBlur(),
            transforms.Transpose(),
            OneOf([
                crop.RandomCrop(224, 224, p=1),
                crop.CenterCrop(224, 224, p=1)
            ], p=0.2),
            resize.Resize(384, 384)
        ], p=0.5)

In [10]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logsRA_new_bottleneck_5_block_b0/lan1')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 30
train_save = 'BiRAFPN_new_bottleneck_5_block_b0'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/image/*')
train_mask_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/mask/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)#train_transform
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b0',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                   compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b0.pth').cuda()

# ---- flops and params ----
params = model.parameters()
optimizer = torch.optim.Adam(params, init_lr)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                    T_max=len(train_loader)*num_epochs,
                                    eta_min=init_lr/1000)



start_epoch = 1

ckpt_path = ''
if ckpt_path != '':
    log = pd.read_csv(ckpt_path.replace('last.pth', 'log.csv'))
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler'])
    optimizer.load_state_dict(checkpoint['optimizer'])

print("#"*20, f"Start Training", "#"*20)
for epoch in range(start_epoch, num_epochs+1):
    #inference(model,writer,epoch)
    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
            train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
    ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
    log = log.append(log_tmp, ignore_index=True)
    log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
    writer.add_scalar('training loss',
                            train_log['loss'].item(),
                            epoch )
    writer.add_scalar('training dice',
                            train_log['dice'].item(),
                            epoch )
    writer.add_scalar('training iou',
                            train_log['iou'].item(),
                            epoch )
    
    if epoch >= num_epochs-20:
        inference(model,writer,epoch)


2022-09-15 09:56:10,149 - mmseg - INFO - Use load_from_local loader
2022-09-15 09:56:10,163 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



#################### Start Training ####################


/home/s/anaconda3/envs/tf2.6/lib/python3.6/site-packages/torch/nn/functional.py:3509: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/home/s/anaconda3/envs/tf2.6/lib/python3.6/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


2022-09-15 09:57:36.813363 Training Epoch [001/030], [loss: 2.7168, dice: 0.7264, iou: 0.5775]
[Saving Checkpoint:] ./snapshots/BiRAFPN_new_bottleneck_5_block_b0/last.pth
2022-09-15 09:59:01.128420 Training Epoch [002/030], [loss: 1.9316, dice: 0.7979, iou: 0.6703]
[Saving Checkpoint:] ./snapshots/BiRAFPN_new_bottleneck_5_block_b0/last.pth
2022-09-15 10:00:25.752349 Training Epoch [003/030], [loss: 1.6617, dice: 0.7959, iou: 0.6705]
[Saving Checkpoint:] ./snapshots/BiRAFPN_new_bottleneck_5_block_b0/last.pth
2022-09-15 10:01:50.760912 Training Epoch [004/030], [loss: 1.3979, dice: 0.8507, iou: 0.7458]
[Saving Checkpoint:] ./snapshots/BiRAFPN_new_bottleneck_5_block_b0/last.pth
2022-09-15 10:03:15.706649 Training Epoch [005/030], [loss: 1.3320, dice: 0.8282, iou: 0.7168]
[Saving Checkpoint:] ./snapshots/BiRAFPN_new_bottleneck_5_block_b0/last.pth
2022-09-15 10:04:40.698038 Training Epoch [006/030], [loss: 1.3605, dice: 0.8447, iou: 0.7408]
[Saving Checkpoint:] ./snapshots/BiRAFPN_new_bottl

scores: dice=0.7762286053700231, miou=0.6951132734003582, precision=0.7847644077247729, recall=0.829124208536389
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8566805186121061, miou=0.7887233619586155, precision=0.817950596090381, recall=0.9596941943809881
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7014835319682647, miou=0.6283379645627524, precision=0.6477686660304796, recall=0.8345705907378025
####################
2022-09-15 10:23:29.283066 Training Epoch [016/030], [loss: 0.9417, dice: 0.9158, iou: 0.8489]
[Saving Checkpoint:] ./snapshots/BiRAFPN_new_bottleneck_5_block_b0/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9014626951042912, miou=0.8467423058721074, precision=0.9418439913393775, recall=0.8889208747412829
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-Cl

scores: dice=0.9074120895656064, miou=0.856045207227168, precision=0.9032174576981947, recall=0.9453714254837399
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.7995713439528405, miou=0.7178138593851798, precision=0.8123457832547626, recall=0.8355377039040105
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8867309496327578, miou=0.8184233698351944, precision=0.8541266692403304, recall=0.9537663970176063
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7878108897968465, miou=0.7075969487810937, precision=0.7268492160956257, recall=0.9096980911205149
####################
2022-09-15 10:38:49.163071 Training Epoch [023/030], [loss: 0.7899, dice: 0.9295, iou: 0.8713]
[Saving Checkpoint:] ./snapshots/BiRAFPN_new_bottleneck_5_block_b0/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: K

scores: dice=0.9092325372481576, miou=0.8565585596400206, precision=0.9402296127128282, recall=0.9005520117960001
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9205022851140653, miou=0.8710061367711154, precision=0.9099703632330818, recall=0.9519484085631434
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.8027432770368615, miou=0.7254847728964695, precision=0.808180510033784, recall=0.8554627610327616
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8919585634221441, miou=0.8222876715566204, precision=0.8562888860395537, recall=0.9558692277424891
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7698761584465804, miou=0.69270702149441, precision=0.7146431554906486, recall=0.8963460239083734
####################
2022-09-15 10:54:10.044505 Training Epoch [030/030], [loss: 0.

In [11]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logsRA_new_bottleneck_5_block_b0/lan2')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 30
train_save = 'BiRAFPN_2_new_bottleneck_5_block_b0'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/image/*')
train_mask_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/mask/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b0',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                  compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b0.pth').cuda()

# ---- flops and params ----
params = model.parameters()

optimizer = torch.optim.Adam(params, init_lr)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                    T_max=len(train_loader)*num_epochs,
                                    eta_min=init_lr/1000)



start_epoch = 1

ckpt_path = ''
if ckpt_path != '':
    log = pd.read_csv(ckpt_path.replace('last.pth', 'log.csv'))
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler'])
    optimizer.load_state_dict(checkpoint['optimizer'])

print("#"*20, f"Start Training", "#"*20)
for epoch in range(start_epoch, num_epochs+1):
    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
            train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
    ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
    log = log.append(log_tmp, ignore_index=True)
    log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
    writer.add_scalar('training loss',
                            train_log['loss'].item(),
                            epoch )
    writer.add_scalar('training dice',
                            train_log['dice'].item(),
                            epoch )
    writer.add_scalar('training iou',
                            train_log['iou'].item(),
                            epoch )
    
    if epoch >= num_epochs-20:
        inference(model,writer,epoch)

2022-09-15 10:54:56,746 - mmseg - INFO - Use load_from_local loader
2022-09-15 10:54:56,755 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



#################### Start Training ####################
2022-09-15 10:56:21.877596 Training Epoch [001/030], [loss: 2.7956, dice: 0.7083, iou: 0.5568]
[Saving Checkpoint:] ./snapshots/BiRAFPN_2_new_bottleneck_5_block_b0/last.pth
2022-09-15 10:57:47.131283 Training Epoch [002/030], [loss: 1.7585, dice: 0.8079, iou: 0.6866]
[Saving Checkpoint:] ./snapshots/BiRAFPN_2_new_bottleneck_5_block_b0/last.pth
2022-09-15 10:59:12.606605 Training Epoch [003/030], [loss: 1.5152, dice: 0.8227, iou: 0.7057]
[Saving Checkpoint:] ./snapshots/BiRAFPN_2_new_bottleneck_5_block_b0/last.pth
2022-09-15 11:00:37.895155 Training Epoch [004/030], [loss: 1.5300, dice: 0.8085, iou: 0.6879]
[Saving Checkpoint:] ./snapshots/BiRAFPN_2_new_bottleneck_5_block_b0/last.pth
2022-09-15 11:02:03.104120 Training Epoch [005/030], [loss: 1.2452, dice: 0.8603, iou: 0.7639]
[Saving Checkpoint:] ./snapshots/BiRAFPN_2_new_bottleneck_5_block_b0/last.pth
2022-09-15 11:03:28.462608 Training Epoch [006/030], [loss: 1.2548, dice: 0.85

scores: dice=0.7775166564261724, miou=0.6996455145996263, precision=0.782900112349798, recall=0.8477265294855445
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8639002438428564, miou=0.7983224514280499, precision=0.8321229109555415, recall=0.9579141046476382
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7178506272206061, miou=0.6399546110736216, precision=0.6626128795515216, recall=0.8613349940658951
####################
2022-09-15 11:22:19.757715 Training Epoch [016/030], [loss: 0.9488, dice: 0.9005, iou: 0.8296]
[Saving Checkpoint:] ./snapshots/BiRAFPN_2_new_bottleneck_5_block_b0/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.8970345969808161, miou=0.8401142814178586, precision=0.9234650610826514, recall=0.891937507075542
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-

scores: dice=0.9152817237290888, miou=0.8652104165002393, precision=0.9026750225409481, recall=0.9563227375021227
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.7781408446897312, miou=0.6984201916997809, precision=0.7816562352592027, recall=0.8451303775401888
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8771365802451828, miou=0.8083736307459808, precision=0.8422178725311329, recall=0.9574301250737152
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7501530317757986, miou=0.669707654116514, precision=0.694262994426113, recall=0.8940597677842508
####################
2022-09-15 11:37:41.665462 Training Epoch [023/030], [loss: 0.7684, dice: 0.9360, iou: 0.8820]
[Saving Checkpoint:] ./snapshots/BiRAFPN_2_new_bottleneck_5_block_b0/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: 

scores: dice=0.9092977132768394, miou=0.8564487755297541, precision=0.9467640626361564, recall=0.8952983416406258
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9234764842894991, miou=0.8747693669788007, precision=0.9139532343655187, recall=0.9550287418558534
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.8013828791244261, miou=0.7237714482445426, precision=0.8150755188736382, recall=0.8463479003005844
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8700913512929241, miou=0.8046220168313458, precision=0.8393871383721664, recall=0.9553017281350275
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7682252429253622, miou=0.6885330294985963, precision=0.7247402067437865, recall=0.8710567836227553
####################
2022-09-15 11:53:03.700016 Training Epoch [030/030], [loss:

In [12]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logsRA_new_bottleneck_5_block_b0/lan3')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 30
train_save = 'BiRAFPN_3_new_bottleneck_5_block_b0'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/image/*')
train_mask_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/mask/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b0',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                  compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b0.pth').cuda()

# ---- flops and params ----
params = model.parameters()
optimizer = torch.optim.Adam(params, init_lr)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                    T_max=len(train_loader)*num_epochs,
                                    eta_min=init_lr/1000)



start_epoch = 1

ckpt_path = ''
if ckpt_path != '':
    log = pd.read_csv(ckpt_path.replace('last.pth', 'log.csv'))
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler'])
    optimizer.load_state_dict(checkpoint['optimizer'])

print("#"*20, f"Start Training", "#"*20)
for epoch in range(start_epoch, num_epochs+1):
    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
            train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
    ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
    log = log.append(log_tmp, ignore_index=True)
    log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
    writer.add_scalar('training loss',
                            train_log['loss'].item(),
                            epoch )
    writer.add_scalar('training dice',
                            train_log['dice'].item(),
                            epoch )
    writer.add_scalar('training iou',
                            train_log['iou'].item(),
                            epoch )
    
    if epoch >= num_epochs-20:
        inference(model,writer,epoch)

2022-09-15 11:53:50,485 - mmseg - INFO - Use load_from_local loader
2022-09-15 11:53:50,494 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



#################### Start Training ####################
2022-09-15 11:55:15.363154 Training Epoch [001/030], [loss: 2.7997, dice: 0.7035, iou: 0.5501]
[Saving Checkpoint:] ./snapshots/BiRAFPN_3_new_bottleneck_5_block_b0/last.pth
2022-09-15 11:56:40.665112 Training Epoch [002/030], [loss: 1.7624, dice: 0.7958, iou: 0.6697]
[Saving Checkpoint:] ./snapshots/BiRAFPN_3_new_bottleneck_5_block_b0/last.pth
2022-09-15 11:58:05.985313 Training Epoch [003/030], [loss: 1.5001, dice: 0.8381, iou: 0.7274]
[Saving Checkpoint:] ./snapshots/BiRAFPN_3_new_bottleneck_5_block_b0/last.pth
2022-09-15 11:59:31.386707 Training Epoch [004/030], [loss: 1.5994, dice: 0.7919, iou: 0.6691]
[Saving Checkpoint:] ./snapshots/BiRAFPN_3_new_bottleneck_5_block_b0/last.pth
2022-09-15 12:00:56.598656 Training Epoch [005/030], [loss: 1.3385, dice: 0.8415, iou: 0.7367]
[Saving Checkpoint:] ./snapshots/BiRAFPN_3_new_bottleneck_5_block_b0/last.pth
2022-09-15 12:02:21.966180 Training Epoch [006/030], [loss: 1.1181, dice: 0.88

scores: dice=0.7576609007238351, miou=0.6786540394832005, precision=0.788371171944632, recall=0.793037214839302
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8489665224254761, miou=0.7807805899323648, precision=0.8153136200702502, recall=0.9386949938053839
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7746340966302402, miou=0.6955586341701457, precision=0.7281614127813577, recall=0.8950099788637298
####################
2022-09-15 12:21:13.229518 Training Epoch [016/030], [loss: 0.9102, dice: 0.9245, iou: 0.8623]
[Saving Checkpoint:] ./snapshots/BiRAFPN_3_new_bottleneck_5_block_b0/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9075134211642226, miou=0.8475617811982539, precision=0.9251902002883265, recall=0.9120143186007326
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-

scores: dice=0.916398078956034, miou=0.864761750300319, precision=0.9026399878489825, recall=0.9529697331151831
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.7908239175275988, miou=0.7093061794950232, precision=0.7979121177018371, recall=0.849637425598073
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8698113638601258, miou=0.7977472922457006, precision=0.8233843175681296, recall=0.9647325193663822
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7569689781882482, miou=0.6715855925434716, precision=0.6985198952604799, recall=0.9034796183589866
####################
2022-09-15 12:36:34.735087 Training Epoch [023/030], [loss: 0.7315, dice: 0.9401, iou: 0.8892]
[Saving Checkpoint:] ./snapshots/BiRAFPN_3_new_bottleneck_5_block_b0/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: K

scores: dice=0.9154829791233797, miou=0.8599699307062691, precision=0.9477725318141548, recall=0.9055555331851142
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9120178901570672, miou=0.8630164965506291, precision=0.9045228002852174, recall=0.9489837883938239
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.7958511107039998, miou=0.7176130256512303, precision=0.8183852487185778, recall=0.8280279166540286
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8872098959994046, miou=0.8206122050181824, precision=0.8534404429510091, recall=0.9565993155087618
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7672493124761331, miou=0.6884983309227223, precision=0.7246587937711599, recall=0.8907661522622108
####################
2022-09-15 12:51:55.697198 Training Epoch [030/030], [loss:

In [13]:
## from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logsRA_new_bottleneck_5_block_b0/lan4')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 30
train_save = 'BiRAFPN_4_new_bottleneck_5_block_b0'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/image/*')
train_mask_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/mask/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b0',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                  compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b0.pth').cuda()

# ---- flops and params ----
params = model.parameters()
optimizer = torch.optim.Adam(params, init_lr)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                    T_max=len(train_loader)*num_epochs,
                                    eta_min=init_lr/1000)



start_epoch = 1

ckpt_path = ''
if ckpt_path != '':
    log = pd.read_csv(ckpt_path.replace('last.pth', 'log.csv'))
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler'])
    optimizer.load_state_dict(checkpoint['optimizer'])

print("#"*20, f"Start Training", "#"*20)
for epoch in range(start_epoch, num_epochs+1):
    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
            train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
    ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
    log = log.append(log_tmp, ignore_index=True)
    log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
    writer.add_scalar('training loss',
                            train_log['loss'].item(),
                            epoch )
    writer.add_scalar('training dice',
                            train_log['dice'].item(),
                            epoch )
    writer.add_scalar('training iou',
                            train_log['iou'].item(),
                            epoch )
    
    if epoch >= num_epochs-20:
        inference(model,writer,epoch)

2022-09-15 12:52:42,414 - mmseg - INFO - Use load_from_local loader
2022-09-15 12:52:42,422 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



#################### Start Training ####################
2022-09-15 12:54:07.412811 Training Epoch [001/030], [loss: 2.8782, dice: 0.7136, iou: 0.5651]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new_bottleneck_5_block_b0/last.pth
2022-09-15 12:55:32.625990 Training Epoch [002/030], [loss: 1.9269, dice: 0.7741, iou: 0.6406]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new_bottleneck_5_block_b0/last.pth
2022-09-15 12:56:57.916478 Training Epoch [003/030], [loss: 1.5567, dice: 0.8234, iou: 0.7068]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new_bottleneck_5_block_b0/last.pth
2022-09-15 12:58:23.245341 Training Epoch [004/030], [loss: 1.4886, dice: 0.8370, iou: 0.7255]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new_bottleneck_5_block_b0/last.pth
2022-09-15 12:59:48.700482 Training Epoch [005/030], [loss: 1.2691, dice: 0.8393, iou: 0.7327]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new_bottleneck_5_block_b0/last.pth
2022-09-15 13:01:14.045555 Training Epoch [006/030], [loss: 1.2835, dice: 0.84

scores: dice=0.7729374214288385, miou=0.6917892551090858, precision=0.784066915245923, recall=0.8212397053369813
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.89077661746797, miou=0.8209634022734013, precision=0.8562994393597791, recall=0.9563660541167833
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7555610335322659, miou=0.6703535433546106, precision=0.7079803063892446, recall=0.8689048406658372
####################
2022-09-15 13:20:05.148481 Training Epoch [016/030], [loss: 0.9327, dice: 0.8909, iou: 0.8160]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new_bottleneck_5_block_b0/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9061655884677202, miou=0.8490054232972598, precision=0.947845820551492, recall=0.8899161240436322
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-Cl

scores: dice=0.9110089660031064, miou=0.8624794998128074, precision=0.9078565408831974, recall=0.9293198939242784
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.7924658580568423, miou=0.713262176663556, precision=0.8113211360408082, recall=0.8321160820145939
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.879017213171326, miou=0.8120423151795051, precision=0.847551140483136, recall=0.9535887990531374
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7702120453446022, miou=0.6957379953253751, precision=0.7303959016717354, recall=0.8834105274490108
####################
2022-09-15 13:35:26.681113 Training Epoch [023/030], [loss: 0.8105, dice: 0.9117, iou: 0.8467]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new_bottleneck_5_block_b0/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: K

scores: dice=0.9093798712165718, miou=0.8553701894620754, precision=0.9391827838306418, recall=0.9073621474627496
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9208579402316653, miou=0.8736441217353538, precision=0.912248916369536, recall=0.955387968927172
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.7919556877939029, miou=0.7158001779956137, precision=0.8082601397955095, recall=0.8343027255917428
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8887320195360314, miou=0.8238778787351959, precision=0.8649697772432986, recall=0.9490822386971964
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7896372512932918, miou=0.7112549992003745, precision=0.7473115638082676, recall=0.8999007751861111
####################
2022-09-15 13:50:48.388080 Training Epoch [030/030], [loss: 0

In [14]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logsRA_new_bottleneck_5_block_b0/lan5')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 30
train_save = 'BiRAFPN_5_new_bottleneck_5_block_b0'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/image/*')
train_mask_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/mask/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b0',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                  compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b0.pth').cuda()

# ---- flops and params ----
params = model.parameters()
optimizer = torch.optim.Adam(params, init_lr)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                    T_max=len(train_loader)*num_epochs,
                                    eta_min=init_lr/1000)



start_epoch = 1

ckpt_path = ''
if ckpt_path != '':
    log = pd.read_csv(ckpt_path.replace('last.pth', 'log.csv'))
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler'])
    optimizer.load_state_dict(checkpoint['optimizer'])

print("#"*20, f"Start Training", "#"*20)
for epoch in range(start_epoch, num_epochs+1):
    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
            train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
    ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
    log = log.append(log_tmp, ignore_index=True)
    log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
    writer.add_scalar('training loss',
                            train_log['loss'].item(),
                            epoch )
    writer.add_scalar('training dice',
                            train_log['dice'].item(),
                            epoch )
    writer.add_scalar('training iou',
                            train_log['iou'].item(),
                            epoch )
    
    if epoch >= num_epochs-20:
        inference(model,writer,epoch)

2022-09-15 13:51:35,243 - mmseg - INFO - Use load_from_local loader
2022-09-15 13:51:35,252 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



#################### Start Training ####################
2022-09-15 13:53:00.154141 Training Epoch [001/030], [loss: 2.7595, dice: 0.7030, iou: 0.5506]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new_bottleneck_5_block_b0/last.pth
2022-09-15 13:54:25.392784 Training Epoch [002/030], [loss: 1.8158, dice: 0.7886, iou: 0.6634]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new_bottleneck_5_block_b0/last.pth
2022-09-15 13:55:50.722603 Training Epoch [003/030], [loss: 1.6409, dice: 0.8111, iou: 0.6927]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new_bottleneck_5_block_b0/last.pth
2022-09-15 13:57:15.919281 Training Epoch [004/030], [loss: 1.4337, dice: 0.8298, iou: 0.7178]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new_bottleneck_5_block_b0/last.pth
2022-09-15 13:58:41.121979 Training Epoch [005/030], [loss: 1.2891, dice: 0.8397, iou: 0.7377]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new_bottleneck_5_block_b0/last.pth
2022-09-15 14:00:06.438021 Training Epoch [006/030], [loss: 1.2286, dice: 0.86

scores: dice=0.7705147771750617, miou=0.6915190818864938, precision=0.7785961370723836, recall=0.840040254391378
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8852565755382629, miou=0.8174091620703278, precision=0.8631413462317891, recall=0.9431501525003629
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7179353158911671, miou=0.6439342918856824, precision=0.6586459620664987, recall=0.8832052505114134
####################
2022-09-15 14:18:57.678928 Training Epoch [016/030], [loss: 0.9307, dice: 0.9158, iou: 0.8471]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new_bottleneck_5_block_b0/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9093747924419178, miou=0.8510322213504299, precision=0.9476376373765906, recall=0.8908514373105798
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC

scores: dice=0.9224369405342212, miou=0.874081625249287, precision=0.9123071898867876, recall=0.9563941828638477
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.7818094331890134, miou=0.7021176546922723, precision=0.7982715691438841, recall=0.8165525827033903
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8797798795235249, miou=0.807434437113553, precision=0.8406937801312124, recall=0.9544292003866115
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.779001704202344, miou=0.6986158230298359, precision=0.7244641273432837, recall=0.9022338229193678
####################
2022-09-15 14:34:19.193009 Training Epoch [023/030], [loss: 0.7600, dice: 0.9362, iou: 0.8823]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new_bottleneck_5_block_b0/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: K

scores: dice=0.9120470375627366, miou=0.8566138985524816, precision=0.9435150852402537, recall=0.9020618238797097
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9209649147676203, miou=0.8729958803924125, precision=0.9084499218708508, recall=0.9569473401058967
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.8014168136084201, miou=0.723735604798626, precision=0.8051006607305967, recall=0.8506383710053019
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8566913490050917, miou=0.7878824473693012, precision=0.8191749957199005, recall=0.9575493805208787
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7616569216121382, miou=0.6836098843128721, precision=0.708035749547056, recall=0.8989034182748435
####################
2022-09-15 14:49:40.759783 Training Epoch [030/030], [loss: 0

In [4]:
model = Net2( backbone=dict(
                  type='mit_b3',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                   compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
             compound_coef=4,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b3.pth')
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
from pthflops import count_ops
print(count_parameters(model))
x = torch.rand(1,3,384,384)
#count_ops(model, x,mode='jit')
from fvcore.nn import FlopCountAnalysis
flops = FlopCountAnalysis(model, x)
flops.total()

2022-08-05 17:09:13,387 - mmseg - INFO - Use load_from_local loader
2022-08-05 17:09:13,500 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias

/home/s/thuannh/SegFormer_clean/mmseg/models/backbones/mix_transformer.py:98: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  q = self.q(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
/home/s/thuannh/SegFormer_clean/mmseg/models/backbones/mix_transformer.py:104: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' func

58400676


/home/s/thuannh/SegFormer_clean/mmseg/models/backbones/mix_transformer.py:106: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  kv = self.kv(x).reshape(B, -1, 2, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
/home/s/anaconda3/envs/tf2.6/lib/python3.6/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
Unsupported operator aten::div encountered 62 time(s)
Unsupported operato

23921844804

In [20]:
from torchsummary import summary


summary(model, (3, 384, 384))

Layer (type:depth-idx)                                  Output Shape              Param #
├─mit_b3: 1-1                                           [-1, 64, 96, 96]          --
|    └─OverlapPatchEmbed: 2-1                           [-1, 9216, 64]            --
|    |    └─Conv2d: 3-1                                 [-1, 64, 96, 96]          9,472
|    |    └─LayerNorm: 3-2                              [-1, 9216, 64]            128
|    └─ModuleList: 2                                    []                        --
|    |    └─Block: 3-3                                  [-1, 9216, 64]            314,880
|    |    └─Block: 3-4                                  [-1, 9216, 64]            314,880
|    |    └─Block: 3-5                                  [-1, 9216, 64]            314,880
|    └─LayerNorm: 2-2                                   [-1, 9216, 64]            128
|    └─OverlapPatchEmbed: 2-3                           [-1, 2304, 128]           --
|    |    └─Conv2d: 3-6                 

Layer (type:depth-idx)                                  Output Shape              Param #
├─mit_b3: 1-1                                           [-1, 64, 96, 96]          --
|    └─OverlapPatchEmbed: 2-1                           [-1, 9216, 64]            --
|    |    └─Conv2d: 3-1                                 [-1, 64, 96, 96]          9,472
|    |    └─LayerNorm: 3-2                              [-1, 9216, 64]            128
|    └─ModuleList: 2                                    []                        --
|    |    └─Block: 3-3                                  [-1, 9216, 64]            314,880
|    |    └─Block: 3-4                                  [-1, 9216, 64]            314,880
|    |    └─Block: 3-5                                  [-1, 9216, 64]            314,880
|    └─LayerNorm: 2-2                                   [-1, 9216, 64]            128
|    └─OverlapPatchEmbed: 2-3                           [-1, 2304, 128]           --
|    |    └─Conv2d: 3-6                 